### FRB Search Volume

Given a DM limit of an FRB search, compute the search volume of the ALFABURST survey.

Dependency: NE2001

Adapted from code by Jayanth Chennamangalam and Evan Keane (calcfrbvol.py)

**Assumptions:**
1. All FRBs originate outside the Galaxy
2. There exists a constant electron density in the IGM (Ioka 2003 and Inoue 2004)
3. Distances are cosmological, $d(z) = \frac{c \cdot z}{H_0}$
4. The number density of galaxies is $n_{\textrm{gal}} = \frac{0.01}{\textrm{Mpc}^{3}}$
5. The host galaxy provides a DM of 100
6. Using a constant z to DM relation of $z = \frac{\textrm{DM}}{1200}$
7. FRBs are flat spectrum, standard candles

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy.units
from astropy.coordinates import SkyCoord
import scipy.integrate
import subprocess

%matplotlib inline

In [2]:
# Parameters
dmMax = 10000. # Dispersion measure limit of search in cm^-3 pc
dmHost = 100. # Assume DM=100 component from source, this asusmption is that the source is embedded in a galaxy

cognizeLog = '../data/cognize.pkl'
#gal_cut = 7. # Cut points +/- degrees around the galactic plane
gal_cut = None

nBeams = 7. # number of beams, for ALFA this is 7
beamFWHM = (nBeams * (3.5 / 60.)) * np.pi/180. # beam FWHM in radians
print 'Beam Width: %f deg (%f rad)'%(beamFWHM * 180./np.pi, beamFWHM)

beamArea = np.pi * (beamFWHM/2.)**2. # beam area in steradians
print 'Beam Area: %f deg^2 (%f str)'%(beamArea * ((180./np.pi)**2.), beamArea)

nGalaxies = 1e-2 # number density of galaxies, Mpc^-3

Beam Width: 0.408333 deg (0.007127 rad)
Beam Area: 0.130954 deg^2 (0.000040 str)


In [3]:
# Constants
deg2rad = np.pi / 180.
rad2deg = 180. / np.pi
Gpc32Mpc3 = 1e9 # convert Gpc^3 to Mpc^3

In [4]:
def coMovingDist(z):
    """Co-moving distance in Gpc, Eq. 3 from Lorimer et al. 2013
    z: float, redshift
    """
    c = 299792.458 # km s^-1
    H0 = 68. # km s^-1 Mpc^-1
    OmegaM = 0.32
    OmegaLambda = 0.68
    
    integrand = lambda zp: 1. / np.sqrt(OmegaM * ((1. + zp)**3.) + OmegaLambda)
    dd, err = scipy.integrate.quad(integrand, 0., z)
    return ((c/H0) * dd) * 1e-3

In [5]:
# Load pandas dataframe from cognize log to get (RA, Dec) and integration time
df = pd.read_pickle(cognizeLog)

In [6]:
activeObs = df[df.status==1] # select log of active observations

# NOTE:only using the central beam
# RA in hours, DEC in degrees and flipped
ra0 = activeObs['RA0'].values * (15. * np.pi / 180.) # Hours to radians
dec0 = (activeObs['DEC0'].values * (np.pi / 180.) ) # Degrees to radians
tint = activeObs['tint'].values # integration time in seconds

totalObsTime = np.sum(tint)
print 'Total observation time: %i seconds'%(totalObsTime)

# Convert (RA, Dec) to Galactic (lat, long)
coords = SkyCoord(ra=ra0 * astropy.units.rad, dec=dec0 * astropy.units.rad, frame='icrs')
gall = coords.galactic.l.degree
galb = coords.galactic.b.degree

# cut out a galactic plane strip
galCutObsTime = totalObsTime
if gal_cut is not None:
    idx = np.argwhere(np.abs(galb) > gal_cut)
    gall = gall[idx].flatten()
    galb = galb[idx].flatten()
    tint = tint[idx].flatten()

    galCutObsTime = np.sum(tint)
    print 'Galaxy Cut Observation time: %i seconds'%(galCutObsTime)

Total observation time: 3830640 seconds


In [7]:
# NE2001 Setup
NE2001root = "/home/griffin/projects/NE2001/" # HARDCODE
NE2001bin = NE2001root + "bin.NE2001/NE2001"
NE2001inp = NE2001root + "input.NE2001/"
tailProcs = "| tr -s ' ' | sed 's/^[ \t]*//'"
# Choose some large distance outside the Galaxy, so that NE2001 returns the
# DM for the entire line-of-sight within the Galaxy
distBound = 100. # kpc
dirDist2DM = "-1" # direction: distance to DM
dirDM2Dist = "1" # direction: DM to distance

In [8]:
# initialization
nGalSurvey = 0. # number of galaxies in the survey
volSurvey = 0. # volume of surveyed universe
areaSurvey = 0. # area of sky surveyed
zVals = [] # distance in z calculated from the NE2001 model based on the maximum DM search

#for l,b in zip(gall[:10],galb[:10]):
print 'Pointings:', gall.shape[0]
for cnt,(l,b) in enumerate(zip(gall,galb)):
    if cnt % 1000 == 0: print cnt
    # Calculate the galactic DM for each poiinting
    # build NE2001 command string
    cmd = " ".join(('cd %s;'%NE2001inp, NE2001bin, str(l), str(b), str(distBound), dirDist2DM, tailProcs))
    outputBuffer = subprocess.check_output(cmd, shell=True, universal_newlines=True, cwd=NE2001inp) # run NE2001 command
    cols = outputBuffer.split("\n")[7].split(" ") # get the 7th line
    dmGal = float(cols[0]) # galactic DM
    dmGal = 50.
    
    dmDiff = dmMax - (dmGal + dmHost) # > 0 if there is excess DM outside the host and our galaxy
    if dmDiff > 0.:
        zz = dmDiff / 1200. # calculate the distance in z to the source using the relation DM ~ 1200 z cm^-3 pc
        # for z <= 2 (Lorimer et al. 2007), based on Ioka (2003) and Inoue (2004)
        
        zVals.append(zz)
        
        dd = coMovingDist(zz) # compute the Hubble distance, in Gpc
        
        # Volume, assuming a cone, V = pi r^2 h / 3, where r = dd theta / 2 (small angle approximation), and h = dd
        # this ignores any sensitivity beyond the beam FWHM
        vol = (np.pi * ((dd * (beamFWHM/2.))**2.) * dd) / 3.
        vol *= Gpc32Mpc3 # convert from Gpc^3 to Mpc^3
        nGal = vol * nGalaxies  # number of galaxies in volume
        
        nGalSurvey += nGal
        volSurvey += vol
        areaSurvey += beamArea

zVals = np.array(zVals)
zMin = np.min(zVals)
zMax = np.max(zVals)
zMean = np.mean(zVals)

Pointings: 63844
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000


In [10]:
print "Total time observing = %i s"%(totalObsTime)
#print "Observing time outside the galactic plane (+/-%.1f degrees) = %i s"%(gal_cut, galCutObsTime)
print "Total volume surveyed = %2.3e Gpc^3" % (volSurvey / Gpc32Mpc3), "= %2.3e Mpc^3" % volSurvey
print "Total area surveyed = %2.3e sq. deg." % areaSurvey
print "z (max = %2.3f, min = %2.3f, mean = %2.3f)" % (zMax, zMin, zMean)
print "Total number of galaxies (assuming n = %2.3f" % nGalaxies, "Mpc^-3) = %2.3f" % nGalSurvey
print dd
print vol
print nGal

Total time observing = 3830640 s
Total volume surveyed = 6.309e+02 Gpc^3 = 6.309e+11 Mpc^3
Total area surveyed = 2.547e+00 sq. deg.
z (max = 8.208, min = 8.208, mean = 8.208)
Total number of galaxies (assuming n = 0.010 Mpc^-3) = 6309459400.750
9.05815352418
9882619.19797
98826.1919797
